In [22]:
import pandas as pd

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [23]:
onehot_encoded_train = pd.get_dummies(df_train)
onehot_encoded_test = pd.get_dummies(df_test)

In [24]:
import lightgbm as lgb

label = onehot_encoded_train['exceeds50K']
data = onehot_encoded_train.drop(['exceeds50K'], axis=1)

data, onehot_encoded_test = data.align(onehot_encoded_test, join='inner', axis=1)

In [25]:
data.shape

(24421, 102)

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, stratify=label)

In [27]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=100, n_jobs=0, num_parallel_tree=1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
       validate_parameters=False, verbosity=None)

In [30]:
import numpy as np

ids = list(range(1, df_test.shape[0] + 1))
y = model.predict(onehot_encoded_test)
output = pd.DataFrame({'id': ids, 'prediction': y})
output.to_csv("submission.csv", index=False)

In [36]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = model.predict(X_test)
classification_report(y_test, y_pred, digits=4)

'              precision    recall  f1-score   support\n\n           0     0.8987    0.9410    0.9194      3713\n           1     0.7803    0.6638    0.7174      1172\n\n   micro avg     0.8745    0.8745    0.8745      4885\n   macro avg     0.8395    0.8024    0.8184      4885\nweighted avg     0.8703    0.8745    0.8709      4885\n'

In [33]:
confusion_matrix(y_test, y_pred)

array([[3494,  219],
       [ 394,  778]], dtype=int64)

In [35]:
accuracy_score(y_test, y_pred)

0.8745138178096213